## Audio speech recognition





In [ ]:
# Install tensorflow
# !pip3 install tensorflow


In [6]:
import tensorflow as tf
from tensorflow.keras import layers

# Sample model architecture (RNN-based)
model = tf.keras.Sequential([
  layers.LSTM(128, return_sequences=True),
  layers.LSTM(128, return_sequences=True),
  layers.Dense(20, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_features, train_labels, validation_data=(val_features, val_labels), epochs=num_epochs)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_features, test_labels)
print(f'Test accuracy: {test_acc}')


NameError: name 'train_features' is not defined

In [ ]:
!pip3 install pydub
!pip3 install librosa
!pip3 install matplotlib
!brew install ffmpeg
!pip3 install pydub librosa matplotlib tensorflow scikit-learn


In [44]:
import numpy as np
from pydub import AudioSegment
import librosa
import librosa.display
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import os

In [45]:

# Find the path to the ffprobe binary installed via Homebrew
# ffmpeg_path = os.path.join(os.popen("brew --prefix").read().strip(), "bin", "ffmpeg")
# ffprobe_path = os.path.join(os.popen("brew --prefix").read().strip(), "bin", "ffprobe")

ffmpeg_path = os.path.join("~", "Downloads", "ffmpeg")
ffprobe_path = os.path.join("~", "Downloads", "ffprobe")

# Set the paths for pydub
AudioSegment.converter = ffmpeg_path
AudioSegment.ffmpeg = ffmpeg_path
AudioSegment.ffprobe = ffprobe_path
ffprobe_path

'~/Downloads/ffprobe'

In [22]:
# Run this code below in your terminal to add the path if home brew havent'
!ln -s /opt/homebrew/bin/ffprobe /usr/local/bin/ffprobe

ln: /usr/local/bin/ffprobe: File exists


In [46]:
!export PATH="~/Downloads/bin:$PATH"
!which ffprobe

ffprobe not found


In [47]:

# Configuration parameters
num_mfccs = 13
sampling_rate = 16000
num_classes = 2  # Change based on the number of classes in your task

# Function to load and preprocess audio files
def load_and_preprocess_audio(file_path):
  audio = AudioSegment.from_file(file_path, format="mp3")
  audio = audio.set_frame_rate(sampling_rate).set_channels(1)  # Convert to mono and set sampling rate
  audio_array = np.array(audio.get_array_of_samples())
  
  mfccs = librosa.feature.mfcc(y=audio_array, sr=sampling_rate, n_mfcc=num_mfccs)
  return mfccs

# Function to load and preprocess a dataset of audio files and corresponding labels
def load_and_preprocess_dataset(data):
  X = []
  y = []
  for file_path, label in data:
    mfccs = load_and_preprocess_audio(file_path)
    X.append(mfccs)
    y.append(label)
  return np.array(X), np.array(y)

# Example data (replace with your actual data)
audio_data = [
  ("../training/audio/0.mp3", "pawn g7")
]

# Load and preprocess the dataset
X, y = load_and_preprocess_dataset(audio_data)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Reshape the input data to match the expected input shape of the model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


/Users/salman/Library/Python/3.9/lib/python/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Pawn takes e6")

# Since this is an interactive Jupyter environment, we can use displacy.render here
displacy.render(doc, style='dep')

In [3]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("e6 Pawn takes e7")

# Since this is an interactive Jupyter environment, we can use displacy.render here
displacy.render(doc, style='dep')

In [4]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("e7 attack using e6")

# Since this is an interactive Jupyter environment, we can use displacy.render here
displacy.render(doc, style='dep')

In [6]:
import spacy
from spacy import displacy

text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.serve(doc, style="ent", port=5001)

/Users/salman/Library/Python/3.9/lib/python/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5001 ...



In [2]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)
matcher.add("PERSON", [[{"lower": "barack"}, {"lower": "obama"}]])
doc = nlp("Barack Obama was the 44th president of the United States")

# 1. Return (match_id, start, end) tuples
matches = matcher(doc)
for match_id, start, end in matches:
    # Create the matched span and assign the match_id as a label
    span = Span(doc, start, end, label=match_id)
    print(span.text, span.label_)

# 2. Return Span objects directly
matches = matcher(doc, as_spans=True)
for span in matches:
    print(span.text, span.label_)

/Users/salman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Barack Obama PERSON
Barack Obama PERSON


In [1]:
import spacy
from spacy.language import Language
from spacy.matcher import Matcher
from spacy.tokens import Token

# We're using a component factory because the component needs to be
# initialized with the shared vocab via the nlp object
@Language.factory("html_merger")
def create_bad_html_merger(nlp, name):
    return BadHTMLMerger(nlp.vocab)

class BadHTMLMerger:
    def __init__(self, vocab):
        patterns = [
            [{"ORTH": "<"}, {"LOWER": "br"}, {"ORTH": ">"}],
            [{"ORTH": "<"}, {"LOWER": "br/"}, {"ORTH": ">"}],
        ]
        # Register a new token extension to flag bad HTML
        Token.set_extension("bad_html", default=False)
        self.matcher = Matcher(vocab)
        self.matcher.add("BAD_HTML", patterns)

    def __call__(self, doc):
        # This method is invoked when the component is called on a Doc
        matches = self.matcher(doc)
        spans = []  # Collect the matched spans here
        for match_id, start, end in matches:
            spans.append(doc[start:end])
        with doc.retokenize() as retokenizer:
            for span in spans:
                retokenizer.merge(span)
                for token in span:
                    token._.bad_html = True  # Mark token as bad HTML
        return doc

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("html_merger", last=True)  # Add component to the pipeline
doc = nlp("Hello<br>world!<test <br/> This is a test.")
for token in doc:
    print(token.text, token._.bad_html)

/Users/salman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Hello False
<br> True
world!<test False
<br/> True
This False
is False
a False
test False
. False


In [5]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH": "("}, {"SHAPE": "ddd"}, {"ORTH": ")"}, {"SHAPE": "ddd"},
           {"ORTH": "-", "OP": "?"}, {"SHAPE": "ddd"}]
matcher.add("PHONE_NUMBER", [pattern])

doc = nlp("Call me at (123) 456 789 or (123) 456 789!")
print([t.text for t in doc])
matches = matcher(doc, as_spans=True)
for (span) in matches:
    print(span.text)


['Call', 'me', 'at', '(', '123', ')', '456', '789', 'or', '(', '123', ')', '456', '789', '!']
(123) 456 789
(123) 456 789
